In [24]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML
import re

1. Start by scraping the data from the summary page for Tennessee's 2nd District, which is available at https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N.
    * The data that we want is contained in the "Total Raised and Spent" table.
    * Make a DataFrame showing, for each candidate:
        * the candidate's name
        * the candidate's party
        * state
        * district number
        * whether the candidate was an incumbent
        * whether the candidate won the race
        * the total amount raised by that candidate (as a numeric variable)
        * the total amount spent by the candidate (as a numeric variable)

In [35]:
URL = 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N.'

tn2nd = requests.get(URL)

tn2ndsoup = BS(tn2nd.text)

tn2nd_table = tn2ndsoup.find('table', attrs={'class' : 'DataTable'})

tn2nd_df = (pd.read_html(str(tn2ndsoup.find('table', attrs={'class' : 'DataTable'}))))[0]
#I'm trying to figure out how to get the district number and state into this dataframe. 

In [69]:
#tn2nd_df['state'] =
re.findall(r"\w{2}\d{2}", URL)
#tn2nd_df['district'] = 

['2020', 'TN02']

In [37]:
type(tn2nd_df)

pandas.core.frame.DataFrame

In [38]:
tn2nd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Candidate     2 non-null      object
 1   Raised        2 non-null      object
 2   Spent         2 non-null      object
 3   Cash on Hand  2 non-null      object
 4   Last Report   2 non-null      object
dtypes: object(5)
memory usage: 208.0+ bytes


In [23]:
type(URL)

str

In [ ]:
https://www.opensecrets.org/races/summary?cycle=2020&id=TN01&spec=N
https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N
https://www.opensecrets.org/races/summary?cycle=2020&id=TN03&spec=N

https://www.opensecrets.org/races/summary?cycle=2020&id={'state'}{'district'}&spec=N
    

In [ ]:

#includes DC so there are 51.
district = 
#{n,m}? Quantifier

States_districts= 'https://www.opensecrets.org/races/summary?cycle=2020&id={}{}&spec=N'.format(state, district)


States_districts_dict = {'state': states, 'district': {1,53}} 
#google python zfill 
s.str.zfill(3)

In [53]:
#https://gist.github.com/JeffPaine/3083347 I didn't want to type all this out
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]